In [36]:
import numpy as np
import csv

# Function to read the header
def read_header(file_path):
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
    return header

# Read headers
x_test_header = read_header('dataset/x_test.csv')
y_train_header = read_header('dataset/y_train.csv')
x_train_header = read_header('dataset/x_train.csv')

# Load csvs x_test, y_test and x_train, skipping the header row
x_test = np.genfromtxt('dataset/x_test.csv', delimiter=',', skip_header=1)
y_train = np.genfromtxt('dataset/y_train.csv', delimiter=',', skip_header=1)
x_train = np.genfromtxt('dataset/x_train.csv', delimiter=',', skip_header=1)
x_train_cleaned = x_train.copy()
x_test_cleaned = x_test.copy()


columns_to_drop_by_name = [
    "FMONTH", "IDATE", "IMONTH", "DISPCODE","IDAY", "IYEAR", "SEQNO", "_PSU", "LADULT", "CTELENUM", 
    "PVTRESD1", "COLGHOUS", "STATERES", "CELLFON3", "CTELNUM1", "CELLFON2", "CADULT", 
    "PVTRESD2", "CCLGHOUS", "CSTATE", "LANDLINE", "HHADULT", "NUMHHOL2", "NUMPHON2", 
    "CPDEMO1", "PREGNANT", "BLIND", "EXRACT11", "EXRACT21", "HIVTST6", "HIVTSTD3", 
    "PAINACT2", "QLMENTL2", "QLSTRES2", "QLHLTH2", "CAREGIV1", "CRGVREL1", "CRGVLNG1", 
    "CRGVHRS1", "CRGVPRB1", "CRGVPERS", "CRGVHOUS", "CRGVMST2", "CRGVEXPT", "VIDFCLT2", 
    "VIREDIF3", "VIPRFVS2", "VINOCRE2", "VIEYEXM2", "VIINSUR2", "VICTRCT4", "VIGLUMA2", 
    "VIMACDG2", "CIMEMLOS", "CDHOUSE", "CDASSIST", "CDHELP", "CDSOCIAL", "CDDISCUS", 
    "HOWLONG", "LASTPAP2", "HPLSTTST", "PROFEXAM", "LENGEXAM", "LSTBLDS3", "HADSGCO1", 
    "LASTSIG3", "PCPSAAD2", "PCPSADI1", "PCPSARE1", "PSATIME", "PCPSARS1", "PCPSADE1", 
    "PCDMDECN", "SCNTPAID", "SCNTWRK1", "SXORIENT", "TRNSGNDR", "RCSGENDR", "RCSRLTN2", 
    "CASTHDX2", "CASTHNO2", "QSTVER", "QSTLANG", "EXACTOT1", "EXACTOT2", "_STSTR", 
    "_STRWT", "_RAWRAKE", "_WT2RAKE", "_CHISPNC", "_CRACE1", "_CPRACE", "_CLLCPWT", 
    "_DUALUSE", "_DUALCOR", "_LLCPWT", "_PRACE1", "FEETCHK2", "PAINACT2", "QLMENTL2", 
    "QLSTRES2", "QLHLTH2", "CAREGIV1", "CRGVREL1", "CRGVLNG1", "CRGVHRS1", "CRGVPRB1", 
    "CRGVPERS", "CRGVHOUS", "CRGVMST2", "CRGVEXPT", "VIDFCLT2", "VIREDIF3", "VIPRFVS2", 
    "VINOCRE2", "VIEYEXM2", "VIINSUR2", "VICTRCT4", "VIGLUMA2", "VIMACDG2", "ASTHMAGE", 
    "ASATTACK", "ASERVIST", "ASDRVIST", "ASRCHKUP", "ASACTLIM", "ASYMPTOM", "ASNOSLEP", 
    "ASTHMED3", "ASINHALR", "HPVADVC2", "HPVADSHT", "_AGE_G", "HTIN4", "_CHLDCNT", 
    "_DRNKWEK", "FTJUDA1_", "FRUTDA1_", "BEANDAY_", "GRENDAY_", "ORNGDAY_", "VEGEDA1_", 
    "_MISFRTN", "_MISVEGN", "_FRTRESP", "_VEGRESP", "_FRT16", "_VEG23", "_FRUITEX", 
    "_VEGETEX", "PAMISS1_", "_PA150R2", "_PA300R2", "_PA30021", "_PASTRNG", "_PAREC1", 
    "_PASTAE1", "_LMTACT1", "_LMTWRK1", "_LMTSCL1", "_RFSEAT3", "_RACE", "_RACEG21", 
    "_RACE_G1", "_AGEG5YR", "_AGE65YR"
]

# Convert column names to indices and combine with automatically identified columns
columns_to_drop_by_index = []
for i in range(x_train_cleaned.shape[1]): 
    unique_values = np.unique(x_train_cleaned[:, i][~np.isnan(x_train_cleaned[:, i])]) 
    if len(unique_values) == 1: 
        columns_to_drop_by_index.append(i)  # Append the index

# Combine indices of manually specified columns and automatically detected ones
columns_to_drop_indices = columns_to_drop_by_index + [i for i, col in enumerate(x_train_header) if col in columns_to_drop_by_name]

# Drop these columns in both x_train and x_test
x_train_cleaned = np.delete(x_train_cleaned, columns_to_drop_indices, axis=1)
x_test_cleaned = np.delete(x_test_cleaned, columns_to_drop_indices, axis=1)

# Drop the same columns from the headers
x_train_header_cleaned = [col for i, col in enumerate(x_train_header) if i not in columns_to_drop_indices]
x_test_header_cleaned = [col for i, col in enumerate(x_test_header) if i not in columns_to_drop_indices]




print(x_train.shape)
print(x_train_cleaned.shape)


(328135, 322)
(328135, 183)


In [37]:

# Extract the _STATE column from x_train_cleaned and x_test_cleaned
state_column_index = x_train_header_cleaned.index("_STATE")  # Get the index of the _STATE column
state_train = x_train_cleaned[:, state_column_index]
state_test = x_test_cleaned[:, state_column_index]

# Identify unique states
unique_states = np.unique(state_train)

# Function to create dummy variables
def create_dummy_variables(state_column, unique_states):
    dummy_matrix = np.zeros((state_column.shape[0], unique_states.shape[0]))  # Initialize the matrix
    
    for i, state in enumerate(unique_states):
        dummy_matrix[:, i] = (state_column == state).astype(int)  # Set 1 where the state matches, 0 otherwise
    
    return dummy_matrix

# Create dummy variables for both train and test
dummy_train = create_dummy_variables(state_train, unique_states)
dummy_test = create_dummy_variables(state_test, unique_states)

# Remove the original _STATE column from x_train_cleaned and x_test_cleaned
x_train_cleaned = np.delete(x_train_cleaned, state_column_index, axis=1)
x_test_cleaned = np.delete(x_test_cleaned, state_column_index, axis=1)

# Append the dummy variables to the original dataset
x_train_cleaned = np.hstack((x_train_cleaned, dummy_train))
x_test_cleaned = np.hstack((x_test_cleaned, dummy_test))

# Add dummy variable column names to the headers
dummy_headers = [f"STATE_{int(state)}" for state in unique_states]  # Create dummy headers like "STATE_1", "STATE_2", etc.
x_train_header_cleaned = [col for col in x_train_header_cleaned if col != "_STATE"] + dummy_headers
x_test_header_cleaned = [col for col in x_test_header_cleaned if col != "_STATE"] + dummy_headers



print(x_train_cleaned.shape)


(328135, 235)


In [38]:
#remove columns with more than 50% missing values from x_train_cleaned and x_test_cleaned
missing_values_threshold = 0.5
columns_to_drop = []
for i in range(x_train_cleaned.shape[1]):
    missing_values = np.isnan(x_train_cleaned[:, i]).sum() / x_train_cleaned.shape[0]
    if missing_values > missing_values_threshold:
        columns_to_drop.append(i)

x_train_cleaned = np.delete(x_train_cleaned, columns_to_drop, axis=1)
x_test_cleaned = np.delete(x_test_cleaned, columns_to_drop, axis=1)

# Remove the same columns from the headers
x_train_header_cleaned = [col for i, col in enumerate(x_train_header_cleaned) if i not in columns_to_drop]
x_test_header_cleaned = [col for i, col in enumerate(x_test_header_cleaned) if i not in columns_to_drop]



print(x_train_cleaned.shape)


(328135, 164)


In [39]:

general_health_column_index = x_train_header_cleaned.index("GENHLTH")

# Extract the "General Health" column
gen_health_train = x_train_cleaned[:, general_health_column_index]
gen_health_test = x_test_cleaned[:, general_health_column_index]

# Function to create dummy variables for general health
def create_gen_health_dummy_variables(health_column):
    # Initialize dummy matrix (for values 1 to 5)
    dummy_matrix = np.zeros((health_column.shape[0], 5))
    
    # Fill dummy variables based on the values in the column
    for i in range(1, 6):  # For values 1 to 5
        dummy_matrix[:, i-1] = (health_column == i).astype(int)
    
    # Values 7, 9, and nan are left as 0 in all dummies
    return dummy_matrix

# Create dummy variables for both train and test datasets
dummy_train_gen_health = create_gen_health_dummy_variables(gen_health_train)
dummy_test_gen_health = create_gen_health_dummy_variables(gen_health_test)

# Remove the original "General Health" column from x_train_cleaned and x_test_cleaned
x_train_cleaned = np.delete(x_train_cleaned, general_health_column_index, axis=1)
x_test_cleaned = np.delete(x_test_cleaned, general_health_column_index, axis=1)

# Append the dummy variables to the original dataset
x_train_cleaned = np.hstack((x_train_cleaned, dummy_train_gen_health))
x_test_cleaned = np.hstack((x_test_cleaned, dummy_test_gen_health))

# Update the headers
gen_health_dummy_headers = [f"GENHLTH_{i}" for i in range(1, 6)]
x_train_header_cleaned = [col for col in x_train_header_cleaned if col != "General Health"] + gen_health_dummy_headers
x_test_header_cleaned = [col for col in x_test_header_cleaned if col != "General Health"] + gen_health_dummy_headers


# Check shapes after adding dummy variables
print(x_train_cleaned.shape)
print(x_test_cleaned.shape)


(328135, 168)
(109379, 168)


In [40]:
#in columns PHYSHLTH, MENTHLTH and POORHLTH, make values 88 into 0, and values 77 and  99 into nan
physhlth_column_index = x_train_header_cleaned.index("PHYSHLTH")
menthlth_column_index = x_train_header_cleaned.index("MENTHLTH")
poorhlth_column_index = x_train_header_cleaned.index("POORHLTH")

# Replace values in PHYSHLTH and MENTHLTH and POORHLTH columns
x_train_cleaned[:, physhlth_column_index][x_train_cleaned[:, physhlth_column_index] == 88] = 0
x_train_cleaned[:, physhlth_column_index][x_train_cleaned[:, physhlth_column_index] == 77] = np.nan
x_train_cleaned[:, physhlth_column_index][x_train_cleaned[:, physhlth_column_index] == 99] = np.nan

x_train_cleaned[:, menthlth_column_index][x_train_cleaned[:, menthlth_column_index] == 88] = 0
x_train_cleaned[:, menthlth_column_index][x_train_cleaned[:, menthlth_column_index] == 77] = np.nan
x_train_cleaned[:, menthlth_column_index][x_train_cleaned[:, menthlth_column_index] == 99] = np.nan

x_train_cleaned[:, poorhlth_column_index][x_train_cleaned[:, poorhlth_column_index] == 88] = 0
x_train_cleaned[:, poorhlth_column_index][x_train_cleaned[:, poorhlth_column_index] == 77] = np.nan
x_train_cleaned[:, poorhlth_column_index][x_train_cleaned[:, poorhlth_column_index] == 99] = np.nan

# Replace values in PHYSHLTH and MENTHLTH columns in test dataset
x_test_cleaned[:, physhlth_column_index][x_test_cleaned[:, physhlth_column_index] == 88] = 0
x_test_cleaned[:, physhlth_column_index][x_test_cleaned[:, physhlth_column_index] == 77] = np.nan
x_test_cleaned[:, physhlth_column_index][x_test_cleaned[:, physhlth_column_index] == 99] = np.nan

x_test_cleaned[:, menthlth_column_index][x_test_cleaned[:, menthlth_column_index] == 88] = 0
x_test_cleaned[:, menthlth_column_index][x_test_cleaned[:, menthlth_column_index] == 77] = np.nan
x_test_cleaned[:, menthlth_column_index][x_test_cleaned[:, menthlth_column_index] == 99] = np.nan

x_test_cleaned[:, poorhlth_column_index][x_test_cleaned[:, poorhlth_column_index] == 88] = 0
x_test_cleaned[:, poorhlth_column_index][x_test_cleaned[:, poorhlth_column_index] == 77] = np.nan
x_test_cleaned[:, poorhlth_column_index][x_test_cleaned[:, poorhlth_column_index] == 99] = np.nan


In [41]:


# List of columns to process
columns_to_process = [
    'HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'CHECKUP1',
    'BPHIGH4', 'BLOODCHO', 'CHOLCHK', 'TOLDHI2'
]

# Collect unique values from both datasets to ensure consistency
unique_values_dict = {}
for column_name in columns_to_process:
    # Get indices of the column in train and test headers
    idx_train = x_train_header_cleaned.index(column_name)
    idx_test = x_test_header_cleaned.index(column_name)
    
    # Extract column data
    column_train = x_train_cleaned[:, idx_train]
    column_test = x_test_cleaned[:, idx_test]
    
    # Replace 7, 9, and BLANK (np.nan) with 0
    column_train = np.where(np.isin(column_train, [7, 9]) | np.isnan(column_train), 0, column_train)
    column_test = np.where(np.isin(column_test, [7, 9]) | np.isnan(column_test), 0, column_test)
    
    # Get unique values excluding 0
    unique_values = np.unique(np.concatenate((column_train, column_test)))
    unique_values = unique_values[unique_values != 0]
    
    unique_values_dict[column_name] = unique_values

def create_dummy_variables_for_column(data, headers, column_name, unique_values):
    # Find the index of the column
    idx = headers.index(column_name)
    # Extract the column data
    column_data = data[:, idx]
    
    # Replace 7, 9, and BLANK (np.nan) with 0
    column_data = np.where(np.isin(column_data, [7, 9]) | np.isnan(column_data), 0, column_data)
    
    # Remove the original column
    data = np.delete(data, idx, axis=1)
    del headers[idx]
    
    # Create dummy variables for each unique value (excluding 0)
    for val in unique_values:
        dummy_column = np.where(column_data == val, 1, 0)
        data = np.column_stack((data, dummy_column))
        headers.append(f"{column_name}_{int(val)}")
    
    return data, headers

def process_dataset(data, headers, unique_values_dict):
    # Make copies to avoid modifying the original data
    data = data.copy()
    headers = headers.copy()
    
    for column_name in columns_to_process:
        data, headers = create_dummy_variables_for_column(
            data, headers, column_name, unique_values_dict[column_name]
        )
    return data, headers

# Process x_train_cleaned
x_train_cleaned_processed, x_train_header_cleaned_processed = process_dataset(
    x_train_cleaned, x_train_header_cleaned, unique_values_dict
)

# Process x_test_cleaned
x_test_cleaned_processed, x_test_header_cleaned_processed = process_dataset(
    x_test_cleaned, x_test_header_cleaned, unique_values_dict
)


print(x_train_cleaned_processed.shape)
print(x_test_cleaned.shape)

(328135, 184)
(109379, 168)


In [44]:
# Save cleaned data with headers
with open('dataset/x_train_cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(x_train_header_cleaned_processed)
    writer.writerows(x_train_cleaned_processed)

with open('dataset/x_test_cleaned.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(x_test_header_cleaned_processed)
    writer.writerows(x_test_cleaned_processed)

